In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Folder format:
- ElectricCar/
  - evrp-benchmark-set/ 
  - Statistic/
- The evrp-benchmark-set folder stores all the .evrp file and the Statistic folder holds all the statistic file



# All the function in EVRP

All the code in here are originated from the code in the TR-EVRP-Competition with some modification.

In [ ]:
# Import all the necessary library
import random
import math
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Choose setup

# Initial construction Nearest Neighbour, Clarke Wright Saving or Minimum Spanning Tree (1|2|3)
initial_construction_choice = 3

# VND random or not (1|2)
VND_randomize = 2

# Use AFS reallcation or not (1|2)
use_AFS_operator = 1

In [ ]:
# To see path
see_path = True

## For statistic

In [ ]:
# initialiazes a run for your heuristic
def start_run(r: int):
    random.seed(r)  # Random seed
    init_evals()
    init_current_best()
    print("Run: {} with random seed {}".format(r, r))


# gets an observation of the run for your heuristic
def end_run(r: int):
    get_mean(r - 1, get_current_best())
    print("End of run {} with best solution quality {} total evaluations: {}".format(r, get_current_best(), get_evals()))


# sets the termination conidition for your heuristic
def termination_condition():
    flag = False

    if get_evals() >= TERMINATION:
        flag = True
    return flag

In [ ]:
def open_stats():
    global perf_of_trials
    global perf_filename
    global log_performance
    global MAX_TRIALS

    # Initialize
    perf_of_trials = [0.0] * MAX_TRIALS

    # get problem name
    name = (problem_instance.split('/')[-1]).split('.')[0]

    # initialize and open output files
    perf_filename = "Statistic/{}.txt".format(name)

    # for performance
    try:
        log_performance = open(perf_filename, mode='w', encoding='UTF-8')
    except:
        raise Exception("Can't open file")


# initialize and open output files


def get_mean(r: int, value: float):
    global perf_of_trials
    perf_of_trials[r] = value


def mean(values, size: int):
    m = 0.0
    for i in range(size):
        m += values[i]

    m = m / float(size)
    return m  # mean


def stdev(values, size: int, average: float):
    dev = 0.0

    if size <= 1:
        return 0.0
    for i in range(size):
        dev += (float(values[i]) - average) * (float(values[i]) - average)
    return math.sqrt(dev / float(size - 1))  # standard deviation


def best_of_vector(values, l: int):
    k = 0
    min = values[k]

    for k in range(1, l):
        if values[k] < min:
            min = values[k]

    return min


def worst_of_vector(values, l: int):
    k = 0
    max = values[k]

    for k in range(1, l):
        if values[k] > max:
            max = values[k]

    return max


def close_stats():
    global MAX_TRIALS
    global log_performance
    global perf_of_trials

    # For statistics
    for i in range(MAX_TRIALS):
        log_performance.write("{0:.2f}".format(perf_of_trials[i]))
        log_performance.write("\n")

    perf_mean_value = mean(perf_of_trials, MAX_TRIALS)
    perf_stdev_value = stdev(perf_of_trials, MAX_TRIALS, perf_mean_value)

    log_performance.write("Mean: {0:.5f}".format(perf_mean_value))
    log_performance.write("\n")
    log_performance.write("Std Dev: {0:.5f}".format(perf_stdev_value))
    log_performance.write("\n")
    log_performance.write("Min: {0:.5f}".format(best_of_vector(perf_of_trials, MAX_TRIALS)))
    log_performance.write("\n")
    log_performance.write("Max: {0:.5f}".format(worst_of_vector(perf_of_trials, MAX_TRIALS)))
    log_performance.write("\n")

    log_performance.close()

## For EVRP

In [ ]:
'''
/****************************************************************/
/*Compute and return the euclidean distance of two objects      */
/****************************************************************/
'''


def euclidean_distance(i: int, j: int):
    xd = node_list[i][1] - node_list[j][1]
    yd = node_list[i][2] - node_list[j][2]
    r = math.sqrt(xd * xd + yd * yd)
    return r


'''
/****************************************************************/
/*Compute the distance matrix of the problem instance           */
/****************************************************************/
'''


def compute_distances():
    global distances

    for i in range(ACTUAL_PROBLEM_SIZE):
        for j in range(ACTUAL_PROBLEM_SIZE):
            distances[i][j] = euclidean_distance(i, j)


'''
/****************************************************************/
/*Generate and return a two-dimension array of type double      */
/****************************************************************/
'''


def generate_2D_matrix_double(n: int, m: int):
    matrix = [[0.0 for i in range(n)] for j in range(m)]
    return matrix


'''
/****************************************************************/
/* Read the problem instance and generate the initial object    */
/* vector.                                                      */
/****************************************************************/
'''


def read_problem(filename: str):
    global problem_size
    global MAX_CAPACITY
    global MIN_VEHICLES
    global BATTERY_CAPACITY
    global energy_consumption
    global NUM_OF_STATIONS
    global OPTIMUM
    global NUM_OF_CUSTOMERS
    global node_list
    global ACTUAL_PROBLEM_SIZE
    global TERMINATION
    global distances
    global cust_demand
    global charging_station
    global DEPOT
    global CUSTOMER_START_INDEX
    global STATION_START_INDEX

    keywords = ""

    with open(filename, mode='r', encoding='UTF-8') as file:
        contents = file.readlines()
        for line in contents:
            word = line.split()

            if word[0] == "DIMENSION:":
                problem_size = int(word[1])
            elif word[0] == "CAPACITY:":
                MAX_CAPACITY = int(word[1])
            elif word[0] == "VEHICLES:":
                MIN_VEHICLES = int(word[1])
            elif word[0] == "ENERGY_CAPACITY:":
                BATTERY_CAPACITY = int(word[1])
            elif word[0] == "ENERGY_CONSUMPTION:":
                energy_consumption = float(word[1])
            elif word[0] == "STATIONS:":
                NUM_OF_STATIONS = int(word[1])
                NUM_OF_CUSTOMERS = problem_size - 1
                ACTUAL_PROBLEM_SIZE = problem_size + NUM_OF_STATIONS
            elif word[0] == "OPTIMAL_VALUE:":
                OPTIMUM = float(word[1])
            elif word[0] == "NODE_COORD_SECTION":
                keywords = "NODE_COORD_SECTION"
                distances = generate_2D_matrix_double(ACTUAL_PROBLEM_SIZE, ACTUAL_PROBLEM_SIZE)
                continue
            elif word[0] == "DEMAND_SECTION":
                keywords = "DEMAND_SECTION"
                cust_demand = [0] * ACTUAL_PROBLEM_SIZE
                charging_station = [False] * ACTUAL_PROBLEM_SIZE
                continue
            elif word[0] == "STATIONS_COORD_SECTION":
                keywords = "STATIONS_COORD_SECTION"
                continue
            elif word[0] == "DEPOT_SECTION":
                keywords = "DEPOT_SECTION"
                continue
            elif word[0] == '-1':
                break

            if keywords == "NODE_COORD_SECTION":
                if problem_size != 0:  # problem_size is the number of customers plus the depot
                    # store initial objects
                    node_list.append([int(word[0]) - 1, float(word[1]), float(word[2])])
                else:
                    print("Error in NODE_COORD_SECTION!!!")
                    exit(0)
            elif keywords == "DEMAND_SECTION":
                if problem_size != 0:
                    cust_demand[int(word[0]) - 1] = int(word[1])
                    if CUSTOMER_START_INDEX == 0:
                        CUSTOMER_START_INDEX = int(word[0]) - 1
            elif keywords == "STATIONS_COORD_SECTION":
                if problem_size != 0:
                    charging_station[int(word[0]) - 1] = True
                    if STATION_START_INDEX == 0:
                        STATION_START_INDEX = int(word[0]) - 1
            elif keywords == "DEPOT_SECTION":
                DEPOT = int(word[0]) - 1
                charging_station[int(word[0]) - 1] = True

    TERMINATION = 25000 * ACTUAL_PROBLEM_SIZE
    compute_distances()


'''
/****************************************************************/
/* Returns the solution quality of the solution. Taken as input */
/* an array of node indeces and its length                      */
/****************************************************************/
'''


def fitness_evaluation(routes, size: int):
    global distances
    global current_best
    global best_routes
    global evals

    tour_length = 0.0

    """	
    The format of the solution that this method evaluates is the following
    Node id:  0 - 5 - 6 - 8 - 0 - 1 - 2 - 3 - 4 - 0 - 7 - 0
    Route id: 1 - 1 - 1 - 1 - 2 - 2 - 2 - 2 - 2 - 3 - 3 - 3
    this solution consists of three routes: 
    Route 1: 0 - 5 - 6 - 8 - 0
    Route 2: 0 - 1 - 2 - 3 - 4 - 0
    Route 3: 0 - 7 - 0
    """

    for i in range(size - 1):
        tour_length += distances[routes[i]][routes[i + 1]]

    if tour_length < current_best:
        current_best = tour_length
        best_routes = routes

    # adds complete evaluation to the overall fitness evaluation count
    evals += 1
    return tour_length


'''
/****************************************************************/
/* Outputs the routes of the solution. Taken as input           */
/* an array of node indeces and its length                      */
/****************************************************************/
'''


def print_solution(routes, size: int):
    for i in range(size):
        print(routes[i], sep=', ')
    print()


'''
/****************************************************************/
/* Returns the distance between two points: from and to. Can be */
/* used to evaluate a part of the solution. The fitness         */
/* evaluation count will be proportional to the problem size    */
/****************************************************************/
'''


def get_distance(_from: int, _to: int):
    global evals
    global ACTUAL_PROBLEM_SIZE
    global distances

    # adds partial evaluation to the overall fitness evaluation count
    # It can be used when local search is used and a whole evaluation is not necessary
    evals += (1.0 / ACTUAL_PROBLEM_SIZE)
    return distances[_from][_to]


'''
/****************************************************************/
/* Returns the energy consumed when travelling between two      */
/* points: from and to.                                         */
/****************************************************************/
'''


def get_energy_consumption(_from: int, _to: int):
    global energy_consumption
    global distances

    # DO NOT USE THIS FUNCTION MAKE ANY CALCULATIONS TO THE ROUTE COST
    return energy_consumption * distances[_from][_to]


'''
/****************************************************************/
/* Returns the demand for a specific customer                   */
/* points: from and to.                                         */
/****************************************************************/
'''


def get_customer_demand(customer: int):
    global cust_demand
    return cust_demand[customer]


'''
/****************************************************************/
/* Returns true when a specific node is a charging station;     */
/* and false otherwise                                          */
/****************************************************************/
'''


def is_charging_station(node: int):
    global charging_station

    flag = False
    if charging_station[node]:
        flag = True
    return flag


'''
/****************************************************************/
/* Returns the best solution quality so far                     */
/****************************************************************/
'''


def get_current_best():
    global current_best
    return current_best


def get_current_best_route():
    global best_routes
    return best_routes


'''
/*******************************************************************/
/* Reset the best solution quality so far for a new indepedent run */
/*******************************************************************/
'''


def init_current_best():
    global current_best
    global best_routes
    current_best = float('inf')
    best_routes = []


'''
/****************************************************************/
/* Returns the current count of fitness evaluations             */
/****************************************************************/
'''


def get_evals():
    global evals
    return evals


'''
/****************************************************************/
/* Reset the evaluation counter for a new indepedent run        */
/****************************************************************/
'''


def init_evals():
    global evals
    evals = 0.0


'''
/****************************************************************/
/* Validates the routes of the solution. Taken as input         */
/* an array of node indeces and its length                      */
/****************************************************************/
'''


def check_solution(t, size: int):
    global BATTERY_CAPACITY
    global MAX_CAPACITY
    global DEPOT
    global NUM_OF_CUSTOMERS

    energy_temp = BATTERY_CAPACITY
    capacity_temp = MAX_CAPACITY
    number_of_customers = 0
    distance_temp = 0.0

    for i in range(size - 1):
        _from = t[i]
        _to = t[i + 1]
        capacity_temp -= get_customer_demand(_to)
        energy_temp -= get_energy_consumption(_from, _to)
        distance_temp += get_distance(_from, _to)

        if get_customer_demand(_to) > 0:
            number_of_customers += 1

        if capacity_temp < 0.0:
            print("error: capacity below 0 at customer {}".format(_to))
            print_solution(t, size)
            exit(1)
        if energy_temp < 0.0:
            print("error: energy below 0 from {} to {}".format(_from, _to))
            print_solution(t, size)
            exit(1)
        if _to == DEPOT:
            capacity_temp = MAX_CAPACITY
        if is_charging_station(_to) or _to == DEPOT:
            energy_temp = BATTERY_CAPACITY

    if distance_temp != fitness_evaluation(t, size):
        print("error: check fitness evaluation")
    if number_of_customers != NUM_OF_CUSTOMERS:
        print("error: not visit all the customer")

## For heuristic

The ideal behind this heuristic is heavily inspired by the report *D. Woller, et al 'Electric Vehicle Routing Problem'* (https://cw.fel.cvut.cz/wiki/_media/courses/ko/evrp_kozak.pdf)

### **Metaheuristic**

- This section hold all the function to run the heuristic
- We use meta-heuristic to solve the EVRP problem 
- Meta-heuristic has two characteristics: intensification and diversification 
	- For diversification we use Variable Neighborhood Search (VNS)
	  - In perturbation we randomly use shuffle, invert and reconnect to create a new path, then repair it using SSF
	- For intensification we use (Randomized)Variable Neighborhood Descent as local search
	  - The operator for local search are: exchange, relocation, two-opt, or-opt and AFS reallocation

In [ ]:
def initialize_heuristic():
	pass


# implement your heuristic in this function
def run_heuristic():
	# Construct initial solution
	current_route = initial_construction()
	stop = False

	while not stop:
		perm_route = perturbation(current_route)
		new_route = seperate_sequential_fixing(perm_route)

		new_route, stop = local_search(new_route)

		if fitness_evaluation(new_route, len(new_route)) < fitness_evaluation(current_route, len(current_route)):
			current_route = new_route


def local_search(solution_route: list):
  global TERMINATION
  global evals
  global VND_randomize
  global use_AFS_operator

  i = 0
  stop = False
  current_route = solution_route[:]

  if use_AFS_operator == 1:
    choice = [1, 2, 3, 4, 5]
  else:
    choice = [1, 2, 3, 4]

  if VND_randomize == 1:
    random.shuffle(choice)

  while i < len(choice):
    method = choice[i]

    if method == 1:
      new_route = relocate(current_route, len(current_route))
    elif method == 2:
      new_route = exchange(current_route, len(current_route))
    elif method == 3:
      new_route = two_opt(current_route, len(current_route))
    elif method == 4:
      new_route = or_opt(current_route, len(current_route))
    else:
      new_route = AFS_reallocation(current_route, len(current_route))

    if fitness_evaluation(new_route, len(new_route)) < fitness_evaluation(current_route, len(current_route)):
      current_route = new_route
      i = 0
      if VND_randomize == 1:
        random.shuffle(choice)
    else:
      i += 1

    if evals >= TERMINATION:
      stop = True
      break

  return current_route, stop


def initial_construction():
  global initial_construction_choice

  # 2 phase construction
  if initial_construction_choice == 1:
    initial_solution = nearest_neighbour()
  elif initial_construction_choice == 2:
    initial_solution = clarke_wright_saving()
  else:
    initial_solution = minimum_spanning_tree()

  repaired_solution = seperate_sequential_fixing(initial_solution)

  return repaired_solution


def perturbation(solution_route: list):
	length = len(solution_route)
	routes = []
	new_route = []
	random_index = random.sample(range(length), PERTURBATION_NUM)
	random_index.sort()

	start_index = 0
	end_index = length

	# Get route in solution based on random index
	for p in random_index:
		routes.append(solution_route[start_index:p])
		start_index = p

	routes.append(solution_route[start_index:end_index])

	# Random do nothing, shuffle and invert
	random_choice = ['nothing', 'shuffle', 'invert']

	for route in routes:
		choice = random.choice(random_choice)

		if choice == 'shuffle':
			random.shuffle(route)
		elif choice == 'invert':
			route.reverse()

	list_number = list(range(PERTURBATION_NUM + 1))
	random.shuffle(list_number)

	# Randomly reconnecting route
	for i in list_number:
		new_route += routes[i]

	# Make sure the begin and the end are depot
	if new_route[0] != 0:
		new_route.insert(0, 0)
	if new_route[-1] != 0:
		new_route.append(0)

	# Remove any redundancy in the solution
	new_route = remove_duplicate_node(new_route)

	return new_route


def remove_duplicate_node(solution_route: list):
	index = 0

	while index < len(solution_route) - 1:
		if solution_route[index] == solution_route[index + 1]:
			del solution_route[index + 1]
			continue
		index += 1

	return solution_route


### **Initial construction**

Each of these method below will produce a tour that go to all the customer regard or disregarding the capacity and battery's level constraints. It starts from depot and also ends with depot.

In [ ]:
# Initial construction method
# Source: https://stemlounge.com/animated-algorithms-for-the-traveling-salesman-problem/
def nearest_neighbour():
  global NUM_OF_CUSTOMERS

  nn_solution = []
  un_visited = list(range(1, NUM_OF_CUSTOMERS + 1))

  nn_solution.append(0)

  while True:
    if not un_visited:
      break

    last_node_index = nn_solution[-1]
    current_best = 2**32 - 1
    current_index_route = 0
    current_index = 0

    for index, value in enumerate(un_visited):
      distance = get_distance(last_node_index, value)

      if distance < current_best:
        current_best = distance
        current_index_route = value
        current_index = index

    nn_solution.append(current_index_route)
    del un_visited[current_index]

  nn_solution.append(0)

  return nn_solution

# Clarke Wright Saving
# Source: https://www.hindawi.com/journals/tswj/2013/874349/?fbclid=IwAR2iu67fgOu40-miFNZBhYTsKfkPUvwu2DKJ15wpqvKEZWHn7T4RZTuGes4
def clarke_wright_saving():
  global problem_size

  a = problem_size
  c = np.zeros((a, a))

  s = []
  for i in range(a):
    for j in range(a):
      if i != j:
        c[i][j] = get_distance(i, j)

  for i in range(a):
    for j in range(a):
      if i != j:
        s.append([i, j, c[0][i] + c[j][0] - c[i][j]])

  for i in range(len(s)):
    s.sort(key=lambda x: x[2], reverse=True)
  route = [0]

  size_s = len(s)
  for i in range(0, size_s):
    if s[i][0] not in route:
      route.append(s[i][0])
    if s[i][1] not in route:
      route.append(s[i][1])
  route.append(0)

  return route


# Minimum Spanning Tree
# Source: https://www.geeksforgeeks.org/kruskals-minimum-spanning-tree-algorithm-greedy-algo-2/?fbclid=IwAR0OSTRDZ6J_jFQ5LEHI_B60ZCP6Cx1S9jUB_jYSY6LgGOJHA8Q9DrEV9w0
class Graph:
    def __init__(self, vertices):
        self.V = vertices  # No. of vertices
        self.graph = []  # default dictionary

    # to store graph

    # function to add an edge to graph
    def addEdge(self, u, v, w):
        self.graph.append([u, v, w])

    # A utility function to find set of an element i
    # (uses path compression technique)
    def find(self, parent, i):
        if parent[i] == i:
            return i
        return self.find(parent, parent[i])

    # A function that does union of two sets of x and y
    # (uses union by rank)
    def union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)

        # Attach smaller rank tree under root of
        # high rank tree (Union by Rank)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot

        # If ranks are same, then make one as root
        # and increment its rank by one
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

    # The main function to construct MST using Kruskal's
    # algorithm
    def KruskalMST(self):

        result = []  # This will store the resultant MST

        # An index variable, used for sorted edges
        i = 0

        # An index variable, used for result[]
        e = 0

        # Step 1: Sort all the edges in
        # non-decreasing order of their
        # weight. If we are not allowed to change the
        # given graph, we can create a copy of graph
        self.graph = sorted(self.graph,
                            key=lambda item: item[2])

        parent = []
        rank = []

        # Create V subsets with single elements
        for node in range(self.V):
            parent.append(node)
            rank.append(0)
        route = [0]
        # Number of edges to be taken is equal to V-1
        while e < self.V - 1:

            # Step 2: Pick the smallest edge and increment
            # the index for next iteration
            u, v, w = self.graph[i]
            i = i + 1
            x = self.find(parent, u)
            y = self.find(parent, v)

            # If including this edge does't
            # cause cycle, include it in result
            # and increment the indexof result
            # for next edge
            if x != y:
                e = e + 1
                result.append([u, v, w])
                self.union(parent, rank, x, y)
        # Else discard the edge

        for x, y, w in result:
            if x not in route:
                route.append(x)
            if y not in route:
                route.append(y)
        route.append(0)

        return route


def minimum_spanning_tree():
    global problem_size

    size_of_node = problem_size
    Mst_algo = Graph(size_of_node)
    d = np.zeros((size_of_node, size_of_node))
    for i in range(size_of_node):
        for j in range(size_of_node):
            if i != j:
                d[i][j] = get_distance(i, j)

    for i in range(size_of_node):
        for j in range(size_of_node):
            if i != j:
                Mst_algo.addEdge(i, j, d[i][j])

    return Mst_algo.KruskalMST()

### **Repair method**

In [ ]:
# Repair method
def seperate_sequential_fixing(solution_route: list):
  global cust_demand
  global MAX_CAPACITY
  global BATTERY_CAPACITY
  global DEPOT

  # First phase: check load constraint
  customers = cust_demand
  load_limit = MAX_CAPACITY
  temp_load = 0

  for index, value in enumerate(solution_route):
    if customers[value] <= 0:
      continue
    temp_load += customers[value]

    # If total load of the next customer exceed the limit then insert depot
    if temp_load > load_limit:
      solution_route.insert(index, 0)
      temp_load = 0

  # Second phase: check battery limit
  max_battery = BATTERY_CAPACITY
  temp_battery = max_battery
  add_battery_flag = False
  index = 1

  while index < len(solution_route):
    _from = solution_route[index-1]
    _to = solution_route[index]

    # If current is depot:
    if _from == DEPOT:
      temp_battery = BATTERY_CAPACITY

    # Check if nearest station can be reach from current and prevent infinity loop
    # Check nearest station in current node
    nearest_station = get_nearest_station(_from)
    temp_battery_station = temp_battery - get_energy_consumption(_from, nearest_station)
    # Check nearest station in next node
    nearest_station_next = get_nearest_station(_to)
    temp_battery_next = temp_battery - get_energy_consumption(_from, _to) - get_energy_consumption(_to, nearest_station_next)

    # If current node can't reach next station, then fallback and find next addable station
    if temp_battery_station < 0:
      add_battery_flag = True
      index -= 1
      temp_battery = get_current_battery_consumption(solution_route, index - 1)
      continue
    # If flag to add station is on then add the nearest station to the solution
    elif add_battery_flag:
      solution_route.insert(index, nearest_station)
      temp_battery = BATTERY_CAPACITY
      index += 1
      add_battery_flag = False
      continue
    # If current node is a station and next node can't reach the station then add the station of the
    # next node to break infinity loop
    elif _from == nearest_station and temp_battery_next < 0:
      solution_route.insert(index, nearest_station_next)
      temp_battery = BATTERY_CAPACITY
      index += 1
      add_battery_flag = False
      continue

    temp_battery -= get_energy_consumption(_from, _to)

    # If vehicle can't reach the next customer then add nearest station
    if temp_battery < 0:
      solution_route.insert(index, nearest_station)
      temp_battery = BATTERY_CAPACITY

    index += 1

  return solution_route


def get_nearest_station(node: int):
  global problem_size
  global ACTUAL_PROBLEM_SIZE

  station_start_index = problem_size
  current_best_length = 2**32 - 1
  current_station = 0

  for station in range(station_start_index, ACTUAL_PROBLEM_SIZE):
    distance = get_distance(node, station)

    if distance < current_best_length:
      current_best_length = distance
      current_station = station

  return current_station


# This function will get the battery's level from the start to the current index in the route
def get_current_battery_consumption(solution_route: list, index: int):
  global BATTERY_CAPACITY

  temp_battery = BATTERY_CAPACITY

  for i in range(1, index + 1):
    head = solution_route[i-1]
    tail = solution_route[i]

    if is_charging_station(tail):
      temp_battery = BATTERY_CAPACITY
    else:
      temp_battery -= get_energy_consumption(head, tail)

  return temp_battery

### **Local search operator**

The operators used in the local search: 

<div align="center">
<img src="https://drive.google.com/uc?export=view&id=1yAPK-8Nk_805ClvECictczD_eFaiv6BJ" width="700"/>

Source: Shinya Watanabe and Kazutoshi Sakakibara (2006). A Multiobjectivization Approach for Vehicle Routing Problems. From conference: Evolutionary Multi-Criterion Optimization, 4th International Conference, EMO 2007, Matsushima, Japan, March 5-8, 2007.

https://www.researchgate.net/publication/221228536_A_Multiobjectivization_Approach_for_Vehicle_Routing_Problems
</div>

In [ ]:
# Local search operator
'''
/****************************************************************/
/*                Local search operator                         */
/****************************************************************/
'''
def two_opt(solution_route: list, steps: int):
	best = solution_route[:]
	best_cost = 0

	for i in range(1, steps-2):
		for j in range(i+1, steps):
			if j-i == 1:
				continue
			if cost_two_opt(best[i-1], best[i], best[j-1], best[j]) > best_cost:
				temp = best[:]
				temp[i:j] = temp[j - 1:i - 1:-1]

				if check_solution_heuristic(temp):
					return temp

	return best


def cost_two_opt(id1: int, id2: int, id3: int, id4: int):
	first_cut = get_distance(id1, id2)
	second_cut = get_distance(id3, id4)
	first_add = get_distance(id3, id1)
	second_add = get_distance(id2, id4)

	return (first_cut + second_cut) - (first_add + second_add)


def exchange(solution_route: list, steps: int):
	best = solution_route[:]
	best_cost = 0

	for i in range(1, steps - 2):
		for j in range(i + 1, steps - 1):
			if j - i == 1:
				continue
			if cost_exchange(best[i - 1], best[i], best[i + 1], best[j - 1], best[j], best[j + 1]) > best_cost:
				temp = best[:]
				temp[i], temp[j] = temp[j], temp[i]

				if check_solution_heuristic(temp):
					return temp

	return best


def cost_exchange(id1: int, id2: int, id3: int, id4: int, id5: int, id6: int):
	cut_one = get_distance(id1, id2)
	cut_two = get_distance(id2, id3)
	cut_three = get_distance(id4, id5)
	cut_four = get_distance(id5, id6)

	add_one = get_distance(id1, id5)
	add_two = get_distance(id5, id3)
	add_three = get_distance(id4, id2)
	add_four = get_distance(id2, id6)

	return (cut_one + cut_two + cut_three + cut_four) - (add_one + add_two + add_three + add_four)


def relocate(solution_route: list, steps: int):
	best = solution_route[:]
	best_cost = 0

	for i in range(1, steps - 2):
		for j in range(i + 1, steps):
			if j - i == 1:
				continue
			if cost_relocate(best[i - 1], best[i], best[i + 1], best[j - 1], best[j]) > best_cost:
				temp = best[:]
				temp.insert(j, temp[i])
				del temp[i]

				if check_solution_heuristic(temp):
					return temp

	return best


def cost_relocate(id1: int, id2: int, id3: int, id4: int, id5: int):
	cut_one = get_distance(id1, id2)
	cut_two = get_distance(id2, id3)
	cut_three = get_distance(id4, id5)

	add_one = get_distance(id1, id3)
	add_two = get_distance(id4, id2)
	add_three = get_distance(id2, id5)

	return (cut_one + cut_two + cut_three) - (add_one + add_two + add_three)


def or_opt(solution_route: list, steps: int):
	best = solution_route[:]
	best_cost = 0

	for i in range(1, steps - 3):
		for j in range(i + 2, steps - 2):
			if j - i < 3:
				continue
			if cost_or_opt(best[i - 1], best[i], best[i + 1], best[i + 2],
							best[j - 1], best[j], best[j + 1], best[j + 2]) > best_cost:
				temp = best[:]
				temp[i:i+2], temp[j:j+2] = temp[j:j+2], temp[i:i+2]

				if check_solution_heuristic(temp):
					return temp

	return best


def cost_or_opt(id1: int, id2_first: int, id3_first: int, id4: int, id5: int, id6_second: int, id7_second: int, id8: int):
	cut_one = get_distance(id1, id2_first)
	cut_two = get_distance(id3_first, id4)
	cut_three = get_distance(id5, id6_second)
	cut_four = get_distance(id7_second, id8)

	add_one = get_distance(id1, id6_second)
	add_two = get_distance(id7_second, id4)
	add_three = get_distance(id5, id2_first)
	add_four = get_distance(id3_first, id8)

	return (cut_one + cut_two + cut_three + cut_four) - (add_one + add_two + add_three + add_four)


def AFS_reallocation(solution_route: list, steps: int):
  global DEPOT
  global BATTERY_CAPACITY

  modified = solution_route[:]

  # Remove duplicate node
  modified = remove_duplicate_node(modified)

  # Delete all the station in the route
  index = 0
  while index < len(modified):
    if is_charging_station(modified[index]) and modified[index] != DEPOT:
      del modified[index]
      continue
    index += 1

  modified = seperate_sequential_fixing(modified)

  return modified


'''
/****************************************************************/
/*                Check solution                                */
/****************************************************************/
'''
def check_solution_heuristic(route: list):
  global BATTERY_CAPACITY
  global MAX_CAPACITY
  global DEPOT

  energy_temp = BATTERY_CAPACITY
  capacity_temp = MAX_CAPACITY
  size = len(route)

  for i in range(size - 1):
    _from = route[i]
    _to = route[i + 1]
    capacity_temp -= get_customer_demand(_to)
    energy_temp -= get_energy_consumption(_from, _to)

    if capacity_temp < 0.0:
      return False
    if energy_temp < 0.0:
      return False
    if _to == DEPOT:
      capacity_temp = MAX_CAPACITY
    if is_charging_station(_to) or _to == DEPOT:
      energy_temp = BATTERY_CAPACITY

  return True

## For drawing graph

In [ ]:
def draw_graph(solution_route, node_list, cust_demand, charging_station, ACTUAL_PROBLEM_SIZE: int):
  '''
  solution_route: have the following format [id-1, ...], it is the route of the solution
  node_list: have the following format [[id:int, x:double, y:double], ...], it is position of the node
  cust_demand: have the following format [demand, ...], it is the demand of the customer
  charging_station: [bool, ...] is the recharging station
  ACTUAL_PROBLEM_SIZE: int is the problem size
  '''

  depot = plt.Circle((node_list[0][1], node_list[0][2]), 2, color='red')
  stations = []
  customers = []
  rect_width = rect_height = 2

  # Vẽ hình vuông, hình tròn và đường đi của route
  for i in range(1, ACTUAL_PROBLEM_SIZE):
    if cust_demand[i] > 0:
      customers.append(plt.Circle((node_list[i][1], node_list[i][2]), 0.5, color='blue'))
    elif charging_station[i]:
      stations.append(plt.Rectangle((node_list[i][1] - rect_width/2, node_list[i][2] - rect_height/2), rect_width, rect_height, lw = 1, color='black'))

  plt.axes()
  plt.gca().add_patch(depot)

  for patch in customers:
    plt.gca().add_patch(patch)
  for patch in stations:
    plt.gca().add_patch(patch)

  for i in range(1, len(solution_route)):
    head = [node_list[solution_route[i - 1]][1], node_list[solution_route[i - 1]][2]]
    tail = [node_list[solution_route[i]][1], node_list[solution_route[i]][2]]
    x = [head[0], tail[0]]
    y = [head[1], tail[1]]
    plt.plot(x, y, 'green', lw=1)

  plt.axis('equal')
  plt.show()

# Main code

In [ ]:
'''Initilize global variable'''

# For EVRP
problem_instance = ""     # Path to the problem
node_list = []            # List of nodes with id and x and y coordinates, a node include [id:int, x:double, y:double]
cust_demand = []          # List with id and customer demands
charging_station = []     # List of charging station
distances = [[]]          # Distance matrix
problem_size = 0          # Problem dimension read
energy_consumption = 0.0

DEPOT = 0                 # Depot index (usually 0)
CUSTOMER_START_INDEX = 0
STATION_START_INDEX = 0
NUM_OF_CUSTOMERS = 0      # Number of customers (excluding depot)
ACTUAL_PROBLEM_SIZE = 0   # Total number of customers, charging stations and depot
OPTIMUM = 0               # Problem's optimal solution
NUM_OF_STATIONS = 0       # Number of charging sations
BATTERY_CAPACITY = 0      # Maximum energy of vehicles
MAX_CAPACITY = 0          # Capacity of vehicles
MIN_VEHICLES = 0          # The minimal vehicles to solve this problem

evals = 0.0               # Total number of evaluation
current_best = float('inf')  # The total distance traveled, the smaller the better
best_routes = []          # Current best routes

TERMINATION = 25000 * ACTUAL_PROBLEM_SIZE  # Will terminate if the total evaluation exceeds this number

# For statistic
MAX_TRIALS = 20           # Number of trial 
log_performance = None    # Used to output offline performance and population diversity
perf_filename = ""        # output files
perf_of_trials = []

# For heuristic
PERTURBATION_NUM = 3			# The number of routes that we want to split

In [ ]:
# Reset global variable to run new problem instance
def reset_global():
  global problem_instance
  global node_list
  global cust_demand
  global charging_station
  global distances
  global problem_size
  global energy_consumption

  global DEPOT
  global CUSTOMER_START_INDEX
  global STATION_START_INDEX
  global NUM_OF_CUSTOMERS
  global ACTUAL_PROBLEM_SIZE
  global OPTIMUM
  global NUM_OF_STATIONS
  global BATTERY_CAPACITY
  global MAX_CAPACITY
  global MIN_VEHICLES

  global evals
  global current_best
  global best_routes
  global TERMINATION

  global log_performance
  global perf_filename
  global perf_of_trials


  # For EVRP
  problem_instance = ""     # Path to the problem
  node_list = []            # List of nodes with id and x and y coordinates, a node include [id:int, x:double, y:double]
  cust_demand = []          # List with id and customer demands
  charging_station = []     # List of charging station
  distances = [[]]          # Distance matrix
  problem_size = 0          # Problem dimension read
  energy_consumption = 0.0

  DEPOT = 0                 # Depot index (usually 0)
  CUSTOMER_START_INDEX = 0
  STATION_START_INDEX = 0
  NUM_OF_CUSTOMERS = 0      # Number of customers (excluding depot)
  ACTUAL_PROBLEM_SIZE = 0   # Total number of customers, charging stations and depot
  OPTIMUM = 0               # Problem's optimal solution
  NUM_OF_STATIONS = 0       # Number of charging sations
  BATTERY_CAPACITY = 0      # Maximum energy of vehicles
  MAX_CAPACITY = 0          # Capacity of vehicles
  MIN_VEHICLES = 0          # The minimal vehicles (routes) to solve this problem

  evals = 0.0               # Total number of evaluation
  current_best = float('inf')  # The total distance traveled, the smaller the better
  best_routes = []          # Current best routes
  TERMINATION = 25000 * ACTUAL_PROBLEM_SIZE  # Will terminate if the total evaluation exceeds this number

  # For statistic
  log_performance = None    # Used to output offline performance and population diversity
  perf_filename = ""        # output files
  perf_of_trials = []

In [ ]:
def main(filePath: str):
  global MAX_TRIALS
  global problem_instance
  global node_list
  global cust_demand
  global charging_station
  global ACTUAL_PROBLEM_SIZE

  # Step 1
  reset_global()
  problem_instance = filePath  # pass the .evrp filename as an argument
  read_problem(problem_instance)  # Read EVRP from file

  # Step 2
  open_stats()  # open text files to store the best values from the 20 runs 

  for run in range(1, MAX_TRIALS + 1):
      # Step 3
      start_run(run)

      # Step 4
      while not termination_condition():
          # Execute your heuristic
          run_heuristic()

      best_route = remove_duplicate_node(get_current_best_route())
      check_solution(best_route, len(best_route))

      # Step 5
      end_run(run)  # store the best solution quality for each run

      # Draw graph
      if see_path:
        draw_graph(best_route, node_list, cust_demand, charging_station, ACTUAL_PROBLEM_SIZE)

  # Step 6
  close_stats()  # close text files to calculate the mean result from the 20 runs 

# Run code

In [ ]:
import os 
from natsort import natsorted, ns

# Loop
%cd /content/gdrive/MyDrive/ElectricCar

pathToBenchmark = '/content/gdrive/MyDrive/ElectricCar/evrp-benchmark-set'

listPath = [os.path.join(pathToBenchmark, file) for file in os.listdir(pathToBenchmark)]
listPath = natsorted(listPath, alg=ns.IGNORECASE)
limit_index = 6

for index, filePath in enumerate(listPath):
  if index > limit_index:
    break

  print('Begin running: ' + filePath)
  main(filePath)